In [1]:
import cv2, os
import numpy as np
from tqdm import tqdm
from PIL import Image
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
path = "EHF/"

data = []
for images in tqdm(os.listdir(path)):
    if images.endswith('.jpg'):
        image_path = path + images
        image = cv2.imread(image_path)
        image = cv2.resize(image,(256,256))
        data.append(image)

100%|███████████████████████████████████████████████████████████████████████████████| 604/604 [00:02<00:00, 219.27it/s]


In [3]:
def load_ply_data(ply_pat):
    pcd = o3d.io.read_point_cloud(ply_path)
    vertices = np.asarray(pcd.points)
    min_coords = np.min(vertices, axis=0)
    max_coords = np.max(vertices, axis=0)
    normalized_vertices = (vertices - min_coords) / (max_coords - min_coords)
    image_array = np.zeros((256, 256, 3), dtype=np.uint8)  # Adjust the image size as needed
    for vertex in normalized_vertices:
        x, y, z = vertex
        image_x = int(x * (image_array.shape[1] - 1))
        image_y = int((1 - y) * (image_array.shape[0] - 1))  # Reverse the y-coordinate
        image_array[image_y, image_x, :] = 255 
    image = Image.fromarray(image_array)
    image.save("check.jpg")
    img = cv2.imread("check.jpg")
    return img

In [4]:
meshes = []

for images in tqdm(os.listdir(path)):
    if images.endswith('.ply'):
        ply_path = path + images
        ply_data = load_ply_data(ply_path)
        meshes.append(ply_data)

100%|███████████████████████████████████████████████████████████████████████████████| 604/604 [00:04<00:00, 132.74it/s]


In [5]:
data = np.array(data)#, dtype="float32") / 255.0
meshes = np.array(meshes)

In [6]:
data.shape, meshes.shape

((100, 256, 256, 3), (100, 256, 256, 3))

In [7]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

In [8]:
encoder_inputs = tf.keras.Input(shape=(256, 256, 3))
x = layers.Conv2D(32, 3, activation='relu', strides=2, padding='same')(encoder_inputs)
x = layers.Conv2D(64, 3, activation='relu', strides=2, padding='same')(x)
x = layers.Conv2D(128, 3, activation='relu', strides=2, padding='same')(x)
x = layers.Conv2D(256, 3, activation='relu', strides=2, padding='same')(x)
x = layers.Flatten()(x)
encoded = layers.Dense(512, activation='relu')(x)

# Decoder network
x = layers.Dense(64 * 64 * 32, activation='relu')(encoded)
x = layers.Reshape((64, 64, 32))(x)
x = layers.Conv2DTranspose(64, 3, activation='relu', strides=2, padding='same')(x)
x = layers.Conv2DTranspose(32, 3, activation='relu', strides=2, padding='same')(x)
decoded = layers.Conv2DTranspose(3, 3, activation='sigmoid', padding='same')(x)

# Autoencoder model
model = tf.keras.Model(encoder_inputs, decoded)

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 256)       295168    
                                                                 
 flatten (Flatten)           (None, 65536)             0         
                                                                 
 dense (Dense)               (None, 512)               335549

In [10]:
def cyclic_lr(epoch, lr):
    base_lr = 0.001
    max_lr = 0.01
    step_size = 5
    cycle = tf.floor(1 + epoch / (2 * step_size))
    x = tf.abs(epoch / step_size - 2 * cycle + 1)
    new_lr = base_lr + (max_lr - base_lr) * tf.maximum(0, (1 - x))
    return new_lr

In [11]:
opt = Adam(learning_rate=0.001)
lr_scheduler = LearningRateScheduler(cyclic_lr)

In [12]:
model.compile(optimizer=opt, loss='mse')
model.fit(data, meshes, batch_size=32, epochs=10, callbacks=[lr_scheduler])

Epoch 1/10
4/4 [==============================] - 11s 904ms/step - loss: 5273.7393 - lr: 0.0010
Epoch 2/10
4/4 [==============================] - 2s 474ms/step - loss: 5261.2183 - lr: 0.0028
Epoch 3/10
4/4 [==============================] - 2s 475ms/step - loss: 5260.2739 - lr: 0.0046
Epoch 4/10
4/4 [==============================] - 2s 474ms/step - loss: 5260.2729 - lr: 0.0064
Epoch 5/10
4/4 [==============================] - 2s 474ms/step - loss: 5260.2744 - lr: 0.0082
Epoch 6/10
4/4 [==============================] - 2s 475ms/step - loss: 5260.2739 - lr: 0.0100
Epoch 7/10
4/4 [==============================] - 2s 473ms/step - loss: 5260.2744 - lr: 0.0082
Epoch 8/10
4/4 [==============================] - 2s 474ms/step - loss: 5260.2739 - lr: 0.0064
Epoch 9/10
4/4 [==============================] - 2s 473ms/step - loss: 5260.2739 - lr: 0.0046
Epoch 10/10
4/4 [==============================] - 2s 473ms/step - loss: 5260.2739 - lr: 0.0028


In [13]:
model.save("3dcheck.h5")

In [14]:
image = cv2.imread("1.jpg")
image = cv2.resize(image,(256,256))
image1 = np.expand_dims(image,axis=0)
pred = model.predict(image1)[0]
cv2.imwrite("output.jpg", pred)

True